# Tutorial 4: The temporal model
In tutorials 1-3 we have used simple fully-connected neural networks to predict sources from single time instances of EEG data. To harness the full information within the EEG, however, we can also incorporate multiple time instances at once. 

For time-series data we can use recurrent neural networks (RNNs). A prominent RNN is the long-short-term memory (LSTM) network, which makes use of temporal information in a quite useful manner.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import mne
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import sys; sys.path.insert(0, '../')
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet.forward import create_forward_model, get_info
plot_params = dict(surface='white', hemi='both', verbose=0)

## Forward model
To get started we just create some generic forward model using the esinet.forward module.

In [ ]:
info = get_info()
info['sfreq'] = 100
fwd = create_forward_model(info=info)

## Simulation
Next, we simulate our training data. In order to invoke the LSTM architecture we need to simulate data that have a temporal dimension. This is controlled via the *duration_of_trial* setting as shown below. We set the duration to 0.2 seconds, which together with our sampling rate of 100 Hz yields 20 time points:
```
100 Hz * 0.2 s = 20
```

Note, that for publication-ready inverse solutions you should increase the number of training samples to 100,000.

In [ ]:
n_samples = 10000
settings = dict(duration_of_trial=0.2)
sim = Simulation(fwd, info, verbose=True, settings=settings).simulate(n_samples=n_samples)

## Build & train LSTM network
The neural network class *Net()* is intelligent and recognizes the temporal structure in the simulations.
It will automatically build the LSTM network architecture without further specification.

In [ ]:
net = Net(fwd)
net.fit(sim)

## Evaluate performance
We can now evaluate the performance of our LSTM network using a newly simulated, thus unseen simulated sample.

In [ ]:
%matplotlib qt
settings['duration_of_trial'] = 2
settings['target_snr'] = 4
# Simulate new data
sim_test = Simulation(fwd, info, settings=settings).simulate(1)
idx = 0
# Predict sources
prediction = net.predict(sim_test)

# Plot Ground Truth
brain = sim_test.source_data[idx].plot(**plot_params)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title')

# Plot simulated EEG
evoked = sim_test.eeg_data[idx].average()
evoked.plot()
evoked.plot_topomap(title='Ground Truth')
evoked = util.get_eeg_from_source(sim_test.source_data[idx], fwd, info, tmin=0.)
evoked.plot_topomap(title='Ground Truth Noiseless')


# Plot predicted source
brain = prediction.plot(**plot_params)
brain.add_text(0.1, 0.9, 'esinet', 'title')
# Plot predicted EEG
evoked_esi = util.get_eeg_from_source(prediction, fwd, info, tmin=0.)
evoked_esi.plot()
evoked_esi.plot_topomap()

# Plot eLORETA predicted source
from esinet.minimum_norm import eloreta
leadfield = util.unpack_fwd(fwd)[1]
y_hat = eloreta(sim_test.eeg_data.get_data()[idx, :, :], leadfield)
stc = util.source_to_sourceEstimate(y_hat, fwd, sfreq=100)
# Plot predicted source
brain = stc.plot(**plot_params)
brain.add_text(0.1, 0.9, 'eLORETA', 'title')
# Plot predicted EEG
evoked_elor = util.get_eeg_from_source(stc, fwd, info, tmin=0.)
evoked_elor.plot()
evoked_elor.plot_topomap()
